In [59]:
import pandas as pd 
import numpy as np 
import category_encoders as ce

In [2]:
#First I read the file 
data = pd.read_excel(r"C:\Users\radek\IronHack\IronRadek\Week3\Day4\Lab32\example_data_cleaning.xlsx")

In [3]:
#Then I check how the data looks like using functions head and shape
data.head()

,TransactionID,ClientID,BirthYear,Amount,Profession,Department,Risk
0,1,34982,2013.0,12900,Student,78,Low
1,2,34983,2015.0,1230,barmen,78,High
2,3,34984,1945.0,3450,student,78,Medium
3,4,34985,1923.0,5670,manager,78,Low
4,5,34986,1978.0,7890,hr,78,High


In [4]:
data.shape

(30, 7)

My plan for data cleaning right now is to: 

- describe data 
- check for the missing values 
- deal with the missing values 
- check if there are columns that can be droped 
- check for innocrect values/consitancy (lower/uper case, special characters etc.)
- check for outliers
- encode non numerical data 

In [5]:
data.describe()

,TransactionID,ClientID,BirthYear,Amount,Department
count,30.000000,30.000000,28.000000,3.000000e+01,30.0
mean,15.500000,34993.833333,1968.316327,1.010097e+05,78.0
std,8.803408,8.132876,27.963867,4.342616e+05,0.0
min,1.000000,34982.000000,1923.000000,1.230000e+03,78.0
25%,8.250000,34987.250000,1944.714286,1.233000e+04,78.0
50%,15.500000,34991.500000,1967.000000,1.899000e+04,78.0
75%,22.750000,34999.750000,1988.000000,2.953500e+04,78.0
max,30.000000,35008.000000,2017.000000,2.399090e+06,78.0


In [7]:
#From the description I see that Transaction ID, ClientID columns are ok. In birth year column I should maybe look at the min and max values. In the Amount column there might be some outliers as well. The Department column can be droped as it has the same value for all the clients. 
data = data.drop(['Department'], axis=1)

In [8]:
data.head()

,TransactionID,ClientID,BirthYear,Amount,Profession,Risk
0,1,34982,2013.0,12900,Student,Low
1,2,34983,2015.0,1230,barmen,High
2,3,34984,1945.0,3450,student,Medium
3,4,34985,1923.0,5670,manager,Low
4,5,34986,1978.0,7890,hr,High


Missing values 

In [9]:
#Now I check for missing values
data.isnull().sum()

TransactionID    0
ClientID         0
BirthYear        2
Amount           0
Profession       2
Risk             0
dtype: int64

In [10]:
#I start with missing birth years
null_displ = data[(data['BirthYear'].isnull()==True)]

In [11]:
#I see that there are two Clients with missing birth year, I can check if there was other transction for these clients, which would allow me to get their birth year 
null_displ

,TransactionID,ClientID,BirthYear,Amount,Profession,Risk
7,22,34987,NaN,18990,sailer,High
8,7,34988,NaN,12330,Manager,Medium


In [14]:
data[(data['ClientID']==34987)]

,TransactionID,ClientID,BirthYear,Amount,Profession,Risk
5,6,34987,1967.0,10110,Manager,Medium
6,15,34987,1967.0,30090,Manager,Low
7,22,34987,NaN,18990,sailer,High


In [15]:
data[(data['ClientID']==34988)]

,TransactionID,ClientID,BirthYear,Amount,Profession,Risk
8,7,34988,NaN,12330,Manager,Medium
9,23,34988,1999.0,25650,Manager,Low


In [16]:
#Now I can manually insert them 
data.at[8,'BirthYear']=1999

In [18]:
data.at[7, 'BirthYear']=1967

In [19]:
#Now I check if there are still missing values in BirthYear column
data[(data['BirthYear'].isnull()==True)]

,TransactionID,ClientID,BirthYear,Amount,Profession,Risk,Birthyear


In [13]:
#The two other missing values were present in the Profession column I will take a look at them 
null_displ = data[(data['Profession'].isnull()==True)]
null_displ

,TransactionID,ClientID,BirthYear,Amount,Profession,Risk
28,28,35008,1967.0,46370,NaN,High
29,29,35008,1976.0,50810,NaN,Medium


In [20]:
#I see that the missing values are for the same client. The client has a diffrent boirthyear which is a mistake but maybe I can verify that if there is another record for that client
data[(data['ClientID']==35008)]

,TransactionID,ClientID,BirthYear,Amount,Profession,Risk,Birthyear
27,27,35008,1967.0,41930,bdm,Low,NaN
28,28,35008,1967.0,46370,NaN,High,NaN
29,29,35008,1976.0,50810,NaN,Medium,NaN


In [21]:
#I see that his correct year of birth is 1967 so I can change that. 
data.at[29, 'BirthYear']=1967

In [22]:
data[(data['ClientID']==35008)]

,TransactionID,ClientID,BirthYear,Amount,Profession,Risk,Birthyear
27,27,35008,1967.0,41930,bdm,Low,NaN
28,28,35008,1967.0,46370,NaN,High,NaN
29,29,35008,1967.0,50810,NaN,Medium,NaN


In [23]:
#Now I can fill these two missing values with profession bdm 
data[['Profession']] = data[['Profession']].fillna('bdm')

In [24]:
#I check if everuthing worked properly
data[(data['ClientID']==35008)]

,TransactionID,ClientID,BirthYear,Amount,Profession,Risk,Birthyear
27,27,35008,1967.0,41930,bdm,Low,NaN
28,28,35008,1967.0,46370,bdm,High,NaN
29,29,35008,1967.0,50810,bdm,Medium,NaN


In [27]:
#Now I should not have any missing values 
data.isnull().sum()

TransactionID    0
ClientID         0
BirthYear        0
Amount           0
Profession       0
Risk             0
dtype: int64

Check for spelling etc 


In [28]:
#I have already droped the column that contained the same information for all of the transations (Department), so now I can check for inconsistancy i.e. Lower upper case etc. I start with the profession column
print(set(data['Profession']))


{'Student', 'student', 'BDM', 'hr', 'sailer', 'developer', 'bdm', 'Manager', 'etudient', 'professor', 'barmen', 'Driver', 'researcher', 'manager', 'Hairdresser', 'HR'}


In [31]:
#I see that there are valeus that use upper/lower case> I also see that there is a student and etudient , so I will change it 
data['Profession'] = data['Profession'].str.lower()
data['Profession'] = data['Profession'].str.replace('etudient', 'student')
print(set(data['Profession'])) 

{'student', 'hr', 'hairdresser', 'sailer', 'driver', 'bdm', 'developer', 'professor', 'barmen', 'researcher', 'manager'}


In [38]:
#Now I do the same with the 'Risk' column
print(set(data['Risk']))
#Since there are only 3 values I can leav it like that

{'Low', 'High', 'Medium'}


Otliers

In [32]:
#I check for extrime values and outliers
stats = data.describe().transpose()

stats

,count,mean,std,min,25%,50%,75%,max
TransactionID,30.0,15.500000,8.803408,1.0,8.250000,15.5,22.75,30.0
ClientID,30.0,34993.833333,8.132876,34982.0,34987.250000,34991.5,34999.75,35008.0
BirthYear,30.0,1968.995238,27.533985,1923.0,1945.892857,1967.0,1988.00,2017.0
Amount,30.0,101009.666667,434261.568869,1230.0,12330.000000,18990.0,29535.00,2399090.0


In [33]:
stats['IQR'] = stats['75%'] - stats['25%']
stats

,count,mean,std,min,25%,50%,75%,max,IQR
TransactionID,30.0,15.500000,8.803408,1.0,8.250000,15.5,22.75,30.0,14.500000
ClientID,30.0,34993.833333,8.132876,34982.0,34987.250000,34991.5,34999.75,35008.0,12.500000
BirthYear,30.0,1968.995238,27.533985,1923.0,1945.892857,1967.0,1988.00,2017.0,42.107143
Amount,30.0,101009.666667,434261.568869,1230.0,12330.000000,18990.0,29535.00,2399090.0,17205.000000


In [34]:
outliers = pd.DataFrame(columns=data.columns)

for col in stats.index:
    iqr = stats.at[col,'IQR']
    cutoff = iqr * 1.5
    lower = stats.at[col,'25%'] - cutoff
    upper = stats.at[col,'75%'] + cutoff
    results = data[(data[col] < lower) | 
                   (data[col] > upper)].copy()
    results['Outlier'] = col
    outliers = outliers.append(results)


In [35]:
#According to the procedure proposed in the lesson. there is one outlier that I can drop 
outliers

,TransactionID,ClientID,BirthYear,Amount,Profession,Risk,Outlier
19,16,34997,1923.0,2399090,developer,High,Amount


In [36]:
data = data.drop(19, axis=0)

In [37]:
data

,TransactionID,ClientID,BirthYear,Amount,Profession,Risk
0,1,34982,2013.000000,12900,student,Low
1,2,34983,2015.000000,1230,barmen,High
2,3,34984,1945.000000,3450,student,Medium
3,4,34985,1923.000000,5670,manager,Low
4,5,34986,1978.000000,7890,hr,High
5,6,34987,1967.000000,10110,manager,Medium
6,15,34987,1967.000000,30090,manager,Low
7,22,34987,1967.000000,18990,sailer,High
8,7,34988,1999.000000,12330,manager,Medium
9,23,34988,1999.000000,25650,manager,Low


In [39]:
#I will take a look at stats again 
stats

,count,mean,std,min,25%,50%,75%,max,IQR
TransactionID,30.0,15.500000,8.803408,1.0,8.250000,15.5,22.75,30.0,14.500000
ClientID,30.0,34993.833333,8.132876,34982.0,34987.250000,34991.5,34999.75,35008.0,12.500000
BirthYear,30.0,1968.995238,27.533985,1923.0,1945.892857,1967.0,1988.00,2017.0,42.107143
Amount,30.0,101009.666667,434261.568869,1230.0,12330.000000,18990.0,29535.00,2399090.0,17205.000000


In [40]:
#I see that also data with birthyear 2017 looks suspecious. I will have a look at it 
data[(data['BirthYear']==2017)]

,TransactionID,ClientID,BirthYear,Amount,Profession,Risk
17,13,34994,2017.0,25650,driver,Medium


In [41]:
#It is not probable that someone who has max 4 years works as a driver. Since there is only 1 transaction for this ClientID I will drop that record as well
data[(data['ClientID']==34994)]

,TransactionID,ClientID,BirthYear,Amount,Profession,Risk
17,13,34994,2017.0,25650,driver,Medium


In [42]:
data = data.drop(17, axis=0)

In [43]:
data

,TransactionID,ClientID,BirthYear,Amount,Profession,Risk
0,1,34982,2013.000000,12900,student,Low
1,2,34983,2015.000000,1230,barmen,High
2,3,34984,1945.000000,3450,student,Medium
3,4,34985,1923.000000,5670,manager,Low
4,5,34986,1978.000000,7890,hr,High
5,6,34987,1967.000000,10110,manager,Medium
6,15,34987,1967.000000,30090,manager,Low
7,22,34987,1967.000000,18990,sailer,High
8,7,34988,1999.000000,12330,manager,Medium
9,23,34988,1999.000000,25650,manager,Low


In [44]:
data.describe()

,TransactionID,ClientID,BirthYear,Amount
count,28.000000,28.000000,28.000000,28.000000
mean,15.571429,34993.714286,1968.923469,21626.785714
std,9.110143,8.405717,25.506227,14416.975348
min,1.000000,34982.000000,1923.000000,1230.000000
25%,7.750000,34987.000000,1947.678571,11775.000000
50%,16.000000,34991.000000,1967.000000,17880.000000
75%,23.250000,35000.250000,1988.000000,28425.000000
max,30.000000,35008.000000,2015.000000,55250.000000


In [45]:
#Now still the person with max Birth year looks suspecious
data[(data['BirthYear']==2015)]


,TransactionID,ClientID,BirthYear,Amount,Profession,Risk
1,2,34983,2015.0,1230,barmen,High


In [46]:
data[(data['ClientID']==34983)]

,TransactionID,ClientID,BirthYear,Amount,Profession,Risk
1,2,34983,2015.0,1230,barmen,High


In [47]:
#It is not probable that someone who is underaged works as a barmen. I will drop that record as well. 
data = data.drop(1, axis=0)

In [48]:
data.describe()

,TransactionID,ClientID,BirthYear,Amount
count,27.000000,27.000000,27.000000,27.000000
mean,16.074074,34994.111111,1967.216931,22382.222222
std,8.879215,8.294267,24.308631,14115.581225
min,1.000000,34982.000000,1923.000000,3450.000000
25%,8.500000,34987.500000,1946.785714,12330.000000
50%,17.000000,34991.000000,1967.000000,18990.000000
75%,23.500000,35000.500000,1988.000000,28980.000000
max,30.000000,35008.000000,2013.000000,55250.000000


In [49]:
#Someone born in 2013 can be a student 
data[(data['BirthYear']==2013)]

,TransactionID,ClientID,BirthYear,Amount,Profession,Risk
0,1,34982,2013.0,12900,student,Low


In [50]:
#These two records look suspecies but lets say that someone below 100 if they are in good condition they can still work. 
data[(data['BirthYear']==1923)]

,TransactionID,ClientID,BirthYear,Amount,Profession,Risk
3,4,34985,1923.0,5670,manager,Low
25,25,35006,1923.0,33050,hr,High


Encoding

In [ ]:
#Now I can encode two columns that are not numerical data i.e. Profession and Risk. For profession column I can use one hot encoding, while for the Risk column I can use ordinal encding as risk can be ordered.

In [52]:
#I start with Profession column
profession_encode=pd.get_dummies(data['Profession'])

In [55]:
#Now I add these results to data df

result = pd.concat([data, profession_encode], axis=1)

In [56]:
result.head(5)

,TransactionID,ClientID,BirthYear,Amount,Profession,Risk,barmen,bdm,hairdresser,hr,manager,professor,researcher,sailer,student
0,1,34982,2013.0,12900,student,Low,0,0,0,0,0,0,0,0,1
2,3,34984,1945.0,3450,student,Medium,0,0,0,0,0,0,0,0,1
3,4,34985,1923.0,5670,manager,Low,0,0,0,0,1,0,0,0,0
4,5,34986,1978.0,7890,hr,High,0,0,0,1,0,0,0,0,0
5,6,34987,1967.0,10110,manager,Medium,0,0,0,0,1,0,0,0,0


In [57]:
#now I can drop profession column
result = result.drop(['Profession'], axis = 1)

In [58]:
result.head(5)

,TransactionID,ClientID,BirthYear,Amount,Risk,barmen,bdm,hairdresser,hr,manager,professor,researcher,sailer,student
0,1,34982,2013.0,12900,Low,0,0,0,0,0,0,0,0,1
2,3,34984,1945.0,3450,Medium,0,0,0,0,0,0,0,0,1
3,4,34985,1923.0,5670,Low,0,0,0,0,1,0,0,0,0
4,5,34986,1978.0,7890,High,0,0,0,1,0,0,0,0,0
5,6,34987,1967.0,10110,Medium,0,0,0,0,1,0,0,0,0


In [68]:
#Now I can encode Risk column using Ordinal encoder 

encoder = ce.OrdinalEncoder(result['Risk'],return_df=True, mapping=[{'col':'Risk', 'mapping':{'High':3,'Low':1,'Medium':2}}])


In [71]:
result['Risk_encoded']=encoder.fit_transform(result['Risk'])

In [72]:
result.head()

,TransactionID,ClientID,BirthYear,Amount,Risk,barmen,bdm,hairdresser,hr,manager,professor,researcher,sailer,student,Risk_encoded
0,1,34982,2013.0,12900,Low,0,0,0,0,0,0,0,0,1,1
2,3,34984,1945.0,3450,Medium,0,0,0,0,0,0,0,0,1,2
3,4,34985,1923.0,5670,Low,0,0,0,0,1,0,0,0,0,1
4,5,34986,1978.0,7890,High,0,0,0,1,0,0,0,0,0,3
5,6,34987,1967.0,10110,Medium,0,0,0,0,1,0,0,0,0,2


In [74]:
#now I can drop column 'Risk'
result = result.drop(['Risk'], axis=1)

Saving cleaned data 

In [75]:
result.to_csv(r'C:\Users\radek\IronHack\IronRadek\Week3\Day4\Lab32\data_cleaned.csv')